In [93]:

import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
from openpyxl import load_workbook
import time
import datetime

def forecastImport(con):
    fc = pd.read_excel("UPDATE.xlsx",sheet_name="FORECAST")
    fc['Item Number'] = fc['Item Number'].astype(str)
    fc = pd.merge(fc, con[['Item Number',"Conversion Factor"]], on='Item Number', how='left')
    fc['Conversion Factor'] = fc['Conversion Factor'].fillna(1)
    fc.iloc[:,2:-1]=fc.iloc[:,2:-1].multiply(fc.iloc[:,-1], axis=0)
    fc =  fc.drop("Conversion Factor", axis=1)
    return fc

def oldForecastImport(con):
    ofc = pd.read_excel("UPDATE.xlsx",sheet_name="PASTFORECAST")
    ofc['Item Number'] = ofc['Item Number'].astype(str)
    ofc = pd.merge(ofc, con[['Item Number',"Conversion Factor"]], on='Item Number', how='left')
    ofc['Conversion Factor'] = ofc['Conversion Factor'].fillna(1)
    ofc.iloc[:,2:-1]=ofc.iloc[:,2:-1].multiply(ofc.iloc[:,-1], axis=0)
    ofc =  ofc.drop("Conversion Factor", axis=1)
    return ofc




def forsImport(con):
    fors = pd.read_excel("UPDATE.xlsx", sheet_name="FORS")
    fors =  fors.rename(columns={"2nd Item Number":"Item Number"})
    fors['Item Number'] = fors['Item Number'].astype(str)
    fors = fors.fillna(0)
    fors = pd.merge(fors, con[['Item Number',"Conversion Factor"]], on='Item Number', how='left')
    fors['Conversion Factor'] = fors['Conversion Factor'].fillna(1)
    fors['Safety Stk 164'] = fors['Safety Stk 164'].astype(int) * fors['Conversion Factor']
    fors['Safety Stk 166'] = fors['Safety Stk 166'].astype(int) * fors['Conversion Factor']
    fors['Safety Stk 168'] = fors['Safety Stk 168'].astype(int) * fors['Conversion Factor']
    fors =  fors.drop("Conversion Factor", axis=1)
    return fors
    
def orderFileImport():
    orders = pd.read_excel("FILE.xlsx", sheet_name="OPEN_ORDERS")
    orders = pd.DataFrame(orders)
    orders = orders.rename(columns={"Item\nNumber":"Item Number"})
    orders['Item Number'] = orders['Item Number'].astype(str)
    orders['Sales Reporting Code 03'] = orders['Sales Reporting Code 03'].astype(int)
    orders['KEY2'] = orders['Order\nNumber'].astype(str)+orders['Item Number'].astype(str)
    return orders

def accountFileImport():
    acc = pd.read_excel("FILE.xlsx",sheet_name="ACCOUNT")
    acc = acc.fillna(0)
    acc['SOLDTO_ACC'] = acc['SOLDTO_ACC'].astype(int)
    acc['SalesCode'] = acc['SalesCode'].astype(int)
    acc['RBU'] = acc['RBU'].astype(int)
    acc['KEY1'] = acc['RBU'].astype(str)+"_"+acc['SalesCode'].astype(str)+"_"+acc['SOLDTO_ACC'].astype(str)
    acc = acc.drop_duplicates(subset='KEY1',keep='last')
    acc = acc[['GROUP','DIVISION','RBU','SalesCode','SOLDTO_ACC','SOLDTO_NAME','Customer',"KEY1"]]
    return acc

def budgetFileImport(date):
    budget = pd.read_excel("UPDATE.xlsx", sheet_name = "BUDGET")
    bud = budget[['RBU','Address Number','Item Number - Free Format','Sales Category Code 3',date['Month'].iloc[0]]]
    bud = bud.rename(columns = {"Item Number - Free Format":"Item Number",date['Month'].iloc[0]:"Budget"})
    bud[['SOLDTO_ACC', 'SOLDTO_NAME']] = bud['Address Number'].str.split(pat='-', n=1, expand=True)
    bud['SOLDTO_ACC'] = bud['SOLDTO_ACC'].str.strip()
    bud['SOLDTO_ACC'] = bud['SOLDTO_ACC'].str.strip()
    bud['Sales Category Code 3'] = bud['Sales Category Code 3'].replace(' ', np.nan)
    bud['Sales Category Code 3'] = bud['Sales Category Code 3'].astype(float).astype(pd.Int64Dtype())
    bud['KEY1'] = bud['RBU'].astype(str)+"_"+bud['Sales Category Code 3'].astype(str)+"_"+bud['SOLDTO_ACC'].astype(str)
    bud['Item Number'] = bud['Item Number'].astype(str)
    budx = bud
    bud = bud.groupby(['KEY1'])['Budget'].sum()
    bud = pd.DataFrame(bud)
    bud.reset_index(inplace=True)
    
    return bud, budx

def mtdFileImport(con):
    mtd = pd.read_excel("UPDATE.xlsx", sheet_name="MTD")
    
    mtd = mtd.dropna()
    mtd[['SOLDTO_ACC', 'SOLDTO_NAME']] = mtd['Sold\nTo'].str.split(pat='-', n=1, expand=True)
    mtd['SOLDTO_ACC'] = mtd['SOLDTO_ACC'].str.strip()
    mtd['SOLDTO_NAME'] = mtd['SOLDTO_NAME'].str.strip()

    mtd[['RBU', 'RBU_NAME']] = mtd['BU\nHeader'].str.split(pat='-', n=1, expand=True)
    mtd['RBU'] = mtd['RBU'].str.strip()
    mtd['RBU_NAME'] = mtd['RBU_NAME'].str.strip()

    mtd[['SalesCode', 'Code_name']] = mtd['Sales Cat 03'].str.split(pat='-', n=1, expand=True)
    mtd['SalesCode'] = mtd['SalesCode'].str.strip()
    mtd['Code_name'] = mtd['Code_name'].str.strip()

    mtd['RBU'] = mtd['RBU'].astype(int)
    mtd['SalesCode'] = mtd['SalesCode'].astype(int)
    mtd['Branch/\nPlant'] = mtd['Branch/\nPlant'].astype(int)
    mtd['SOLDTO_ACC'] = mtd['SOLDTO_ACC'].astype(int)

    mtd = mtd.rename(columns={"Item\nNumber":"Item Number","Qty\nShipped\nto Date":"QtyShipped"})
    mtd['Item Number'] = mtd['Item Number'].astype(str)
    mtd = pd.merge(mtd, con[["Item Number","Conversion Factor"]], on="Item Number", how='left')
    mtd['Conversion Factor'] = mtd['Conversion Factor'].fillna(1)
    mtd['QtyShipped'] = mtd['Conversion Factor'].astype(int) * mtd['QtyShipped'].astype(int)

    mtd['KEY1'] = mtd['RBU'].astype(str)+"_"+mtd['SalesCode'].astype(str)+"_"+mtd['SOLDTO_ACC'].astype(str)
    
    mtd['KEY2'] = mtd['Order\nNumber'].astype(str)+mtd['Document\nNumber'].astype(str)+mtd['Line\nNumber'].astype(str)
    
    mtd = mtd.drop_duplicates(subset='KEY2', keep='last')
    
    mtd = mtd[(mtd['SalesCode']!=250)|(mtd['SalesCode']!=500)|(mtd['SalesCode']!=110)]
    
    return mtd

def otherFileImport():
    date = pd.read_excel("File.xlsx",sheet_name="ORDERDATE")
    con = pd.read_excel("FILE.xlsx",sheet_name="CONVERSION")
    return date, con

    
    

date, con = otherFileImport()

orders = orderFileImport()

acc = accountFileImport()

mtd = mtdFileImport(con)
bud, budx = budgetFileImport(date)
fors = forsImport(con)

fc = forecastImport(con)
ofc = oldForecastImport(con)

def pFCExtract(con,ofc):

    pfc = ofc.groupby('Item Number').apply(lambda x: x.iloc[:, 2:5].sum())
    pfc.reset_index(inplace=True)
    return pfc

pfc = pFCExtract(con,ofc)

def lastFC(ofc,fors):
    lfc = pd.merge(ofc,fors[['Item Number','Sls Cd3']], on='Item Number', how='left')
    lfc = lfc.rename(columns={"Sls Cd3":"SalesCode"})
    lfc['SalesCode'] = lfc['SalesCode'].replace(' ', np.nan)
    lfc['SalesCode'] = lfc['SalesCode'].astype(float).astype(pd.Int64Dtype())
    lfc['KEY3'] = lfc['RBU'].astype(str)+"_"+lfc['SalesCode'].astype(str)+"_"+lfc['Item Number'].astype(str)
    
    return lfc
lfc = lastFC(ofc,fors)
   
    
def concatProcess_sales(mtd,acc,bud):
    print("WHYY")
    custSales = mtd.groupby(['KEY1'])['QtyShipped'].sum()
    custSales = pd.DataFrame(custSales)
    custSales.reset_index(inplace=True)
    print("   7.1")
    
    acc = pd.merge(acc, bud, on="KEY1",how='left')
    UnRegisteredBudget = bud[~bud['KEY1'].isin(acc['KEY1'])]
    print("   7.2")
    
    acc = pd.merge(acc,custSales, on="KEY1", how='left')
    acc['QtyShipped'] = acc['QtyShipped'].fillna(0)
    print("   7.3")
    
    UnRegistered = custSales[~custSales['KEY1'].isin(acc['KEY1'])]
    UnRegistered = pd.merge(UnRegistered,mtd[['KEY1','Ship\nTo','SOLDTO_ACC', 'SOLDTO_NAME', 'RBU', 'RBU_NAME', 'SalesCode']], 
                       on="KEY1", how="left")
    print("   7.4")
    
    acc = acc.drop_duplicates(subset='KEY1', keep="last")
    
    return acc, UnRegistered, UnRegisteredBudget

def concatProcess_orders(acc,orders,con):
    orders = pd.merge(orders,con[["Item Number","Conversion Factor"]], on="Item Number", how='left')
    orders = orders.drop_duplicates(subset="KEY2", keep='last')
    orders['Conversion Factor'] = orders['Conversion Factor'].fillna(1)
    
    orders['TotalOpenOrders'] = orders['Qty\nOrder/\nTransaction'].astype(int)*orders['Conversion Factor'].astype(int)
    orders['TotalBackOrders'] = orders['Qty\nBackorder/\nHeld'].astype(int)*orders['Conversion Factor'].astype(int)
    
    def pastDue_Orders(row):
        if row['Requested\nDate'] < date['Beg_Month'].iloc[0]:
            return row['TotalOpenOrders']
        else:
            return 0
    
    def currDue_Orders(row):
        if row['Requested\nDate'] >= date['Beg_Month'].iloc[0] and row['Requested\nDate'] <= date['End_Month'].iloc[0]:
            return row['TotalOpenOrders']
        else:
            return 0

    def postDue_Orders(row):
        if row['Requested\nDate'] > date['End_Month'].iloc[0]:
            return row['TotalOpenOrders']
        else:
            return 0

    # Apply the custom_condition function to create a new column 'C' based on the condition
    orders['PastDue_Orders'] = orders.apply(pastDue_Orders, axis=1)
    orders['CurrDue_Orders'] = orders.apply(currDue_Orders, axis=1)
    orders['PostDue_Orders'] = orders.apply(postDue_Orders, axis=1)
    
    def pastDue_bo(row):
        if row['Requested\nDate'] < date['Beg_Month'].iloc[0]:
            return row['TotalBackOrders']
        else:
            return 0
    
    def currDue_bo(row):
        if row['Requested\nDate'] >= date['Beg_Month'].iloc[0] and row['Requested\nDate'] <= date['End_Month'].iloc[0]:
            return row['TotalBackOrders']
        else:
            return 0

    def postDue_bo(row):
        if row['Requested\nDate'] > date['End_Month'].iloc[0]:
            return row['TotalBackOrders']
        else:
            return 0

    # Apply the custom_condition function to create a new column 'C' based on the condition
    orders['PastDue_BO'] = orders.apply(pastDue_bo, axis=1)
    orders['CurrDue_BO'] = orders.apply(currDue_bo, axis=1)
    orders['PostDue_BO'] = orders.apply(postDue_bo, axis=1)

    orders['KEY1'] = orders['BU\nHeader'].astype(str)+"_"+orders['Sales Reporting Code 03'].astype(str)+"_"+orders['Sold\nTo'].astype(str)
    
    grp = orders.groupby(['KEY1'])[['TotalOpenOrders','TotalBackOrders',"PastDue_Orders","CurrDue_Orders","PostDue_Orders",
                               "PastDue_BO","CurrDue_BO","PostDue_BO"]].sum()
    grp = pd.DataFrame(grp)
    grp.reset_index(inplace=True)
    acc = pd.merge(acc,grp[['KEY1','TotalOpenOrders',"TotalBackOrders",
                           'PastDue_Orders','CurrDue_Orders','PostDue_Orders',
                          "PastDue_BO",'CurrDue_BO','PostDue_BO']], on='KEY1',how='left')
    
    UnRegisteredOrders = orders[~orders['KEY1'].isin(acc['KEY1'])]
    
    acc = acc.drop_duplicates(subset='KEY1', keep="last")
    
    return acc, UnRegisteredOrders, orders
    


acc, UnRegistered, UnRegisteredBudget = concatProcess_sales(mtd,acc,bud)

acc, UnRegisteredOrders, orders = concatProcess_orders(acc,orders,con)


def itemProcess(budx,mtd,orders,fc,acc,fors,pfc):
    budx = budx.rename(columns={'Sales Category Code 3':"SalesCode"})
    budx['KEY3'] = budx['RBU'].astype(str)+"_"+budx['SalesCode'].astype(str)+"_"+budx['Item Number'].astype(str)
    budx = budx.groupby(['KEY3','RBU','SalesCode','Item Number'])['Budget'].sum()
    budx = pd.DataFrame(budx)
    budx.reset_index(inplace=True)
    
    mtd['KEY3'] = mtd['RBU'].astype(str)+"_"+mtd['SalesCode'].astype(str)+"_"+mtd['Item Number'].astype(str)
    mtd = mtd[['KEY3','RBU',"SalesCode","Item Number","QtyShipped"]]
    mtd = mtd.groupby(['KEY3','RBU','SalesCode','Item Number'])['QtyShipped'].sum()
    mtd = pd.DataFrame(mtd)
    mtd.reset_index(inplace=True)
    
    fc = pd.merge(fc,fors[['Item Number','Sls Cd3']],on='Item Number',how='left')
    fc['Sls Cd3'] = fc['Sls Cd3'].replace(' ', np.nan)
    fc['Sls Cd3'] = fc['Sls Cd3'].astype(float).astype(pd.Int64Dtype())
    fc = fc.rename(columns={"Sls Cd3":"SalesCode"})
    fc['KEY3'] = fc['RBU'].astype(str)+"_"+fc['SalesCode'].astype(str)+"_"+fc['Item Number'].astype(str)
    fc = fc.groupby(['KEY3', 'RBU',"SalesCode", 'Item Number']).apply(lambda x: x.iloc[:, 2:5].sum())
    fc.reset_index(inplace=True)
    
    test = pd.merge(budx,fc, on=['KEY3','RBU',"SalesCode",'Item Number'], how='outer')
    test = test.fillna(0)
    
    test = pd.merge(test,mtd,on=['KEY3','RBU',"SalesCode",'Item Number'], how="outer")
    test = test.fillna(0)
    
    orders = orders.rename(columns={'Sales Reporting Code 03':"SalesCode"})
    orders['SalesCode'] = orders['SalesCode'].replace(' ', np.nan)
    orders['SalesCode'] = orders['SalesCode'].astype(float).astype(pd.Int64Dtype())
    
    ordersx = orders[['BU\nHeader',"SalesCode","Item Number",'TotalOpenOrders',"TotalBackOrders",
                  'PastDue_Orders','CurrDue_Orders','PostDue_Orders',
                  "PastDue_BO",'CurrDue_BO','PostDue_BO']]
    ordersx = ordersx.rename(columns = {'BU\nHeader':"RBU"})
    ordersx['KEY3'] = ordersx['RBU'].astype(str)+"_"+ordersx['SalesCode'].astype(str)+"_"+ordersx['Item Number'].astype(str)
    ordersx = ordersx.groupby(['KEY3', 'RBU',"SalesCode",'Item Number'])[['TotalOpenOrders', 'TotalBackOrders',
                                                       'PastDue_Orders', 'CurrDue_Orders',
                                                       'PostDue_Orders', 'PastDue_BO',
                                                       'CurrDue_BO', 'PostDue_BO']].sum()
    ordersx.reset_index(inplace=True)
    
    test = pd.merge(test,ordersx,on=['KEY3','RBU',"SalesCode",'Item Number'], how="outer")
    test = test.fillna(0)
    test = test.drop_duplicates(subset='KEY3', keep='last')
    
    rbu = acc[['GROUP','DIVISION','RBU','SalesCode']]
    rbu = rbu.drop_duplicates(subset=['GROUP','DIVISION','RBU','SalesCode'], keep='last')
    rbu['KEY4'] = rbu['RBU'].astype(str)+"_"+rbu['SalesCode'].astype(str)
    
    #test = pd.merge(test, fors[['Item Number',"Sls Cd3"]], on="Item Number", how='left')
    ##test['Sls Cd3'] = test['Sls Cd3'].replace(' ', np.nan)
    #test['Sls Cd3'] = test['Sls Cd3'].astype(float).astype(pd.Int64Dtype())
    test['KEY4'] = test['RBU'].astype(str)+"_"+test['SalesCode'].astype(str)
    
    test = pd.merge(test,rbu[['KEY4','GROUP','DIVISION']], on='KEY4',how='left')
    test = test.drop_duplicates(subset='KEY3', keep='last')
    
    return test

test = itemProcess(budx,mtd,orders,fc,acc,fors,pfc)

def wo_prep(con):
    wo = pd.read_excel("UPDATE.xlsx",sheet_name="WORKORDER")
    from datetime import datetime, date,timedelta
    wo['[SCHEDULED DATE]'] = pd.to_datetime(wo['[SCHEDULED DATE]'], format='%d-%m-%Y')
    wo['Year'] = wo['[SCHEDULED DATE]'].dt.year
    wo['Year'] = wo['Year'].astype(int)
    wo['Month'] = wo['[SCHEDULED DATE]'].dt.month
    wo['Month'] = wo['Month'].astype(int)
    wo['weekofyear'] = wo['[SCHEDULED DATE]'].dt.isocalendar().week
    wo['weekofyear'] = wo['weekofyear'].astype(int)
    wo['Today'] = date.today()
    wo['TodayW'] = wo['Today'].apply(lambda x:x.isocalendar()[1])
    wo['TodayY'] = pd.to_datetime(wo['Today'])
    wo['TodayY'] =  wo['TodayY'].dt.year
    
    conn = con
    conn = conn.rename(columns={"Item Number":"ITEM NUMBER"})
    wo = pd.merge(wo,conn[['ITEM NUMBER',"Conversion Factor"]], on='ITEM NUMBER',how='left')
    wo['Conversion Factor'] = wo['Conversion Factor'].fillna(1)
    wo['[QTY OUTSTANDING]'] = wo['[QTY OUTSTANDING]'].astype(float)*wo['Conversion Factor'].astype(float)

    conditions = [
        (wo['Year'] < wo['TodayY']),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] < wo['TodayW']),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+1),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+2),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+3),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+4),
        (wo['Year'] == wo['TodayY']) & (wo['weekofyear'] > wo['TodayW']+4),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+1),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+2),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+3),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] == wo['TodayW']+4),
        (wo['Year'] > wo['TodayY']) & (wo['weekofyear'] > wo['TodayW']+4)]

    values = ['ByPass', 'ByPass', 'Current', 'Curr+1','Curr+2','Curr+3','Curr+4','Curr+5','Curr+1','Curr+2','Curr+3','Curr+4','Curr+5']

    
    wo['WO_Status'] = np.select(conditions, values)

    cols = ['ByPass', 'Current', 'Curr+1','Curr+2','Curr+3','Curr+4','Curr+5']

    for i in range(7):

        condition = [wo['WO_Status']== cols[i]]
        val = [wo['[QTY OUTSTANDING]']]

        wo[cols[i]] = np.select(condition,val)
        
    wo = wo.fillna(0)

    wo.rename(columns = {"ITEM NUMBER":"Item Number"}, inplace = True)
    wo['Item Number'] = wo['Item Number'].astype(str)
    wo = wo[["Item Number",'ByPass', 'Current','Curr+1','Curr+2','Curr+3','Curr+4','Curr+5']]
    wo = wo.pivot_table(index = "Item Number", aggfunc= "sum")
    wo = wo.reset_index()
    column_order = ["Item Number", 'ByPass', 'Current', 'Curr+1', 'Curr+2', 'Curr+3', 'Curr+4', 'Curr+5']
    wo = wo[column_order]
    
    
    
    current_date = datetime.now()
    days_until_friday = (4 - current_date.weekday() + 7) % 7
    fridays = []
    for _ in range(8):
        fridays.append("WO " + (current_date + timedelta(days=days_until_friday)).strftime("%d-%b"))
        days_until_friday += 7
    
    wo = wo.rename(columns={"Curr+1":fridays[1],"Curr+2":fridays[2],
                       "Curr+3":fridays[3],"Curr+4":fridays[5],
                       "Curr+5":"WO POSTDUE","ByPass":"WO ByPass",
                           "Current":"WO Current"})
    wo['Item Number'] = wo['Item Number'].astype(str)
    return wo
wo = wo_prep(con)


def sohExtract(con):
    soh = pd.read_excel("UPDATE.xlsx", sheet_name="SOH")
    soh = soh.rename(columns={"Branch/\nPlant":"WHS"})
    soh['Item Number'] = soh['Item\nNumber'].str.rsplit(' ',expand=True)[0]
    soh = pd.merge(soh,con[["Item Number","Conversion Factor"]], on='Item Number',how='left')
    soh['Conversion Factor'] = soh['Conversion Factor'].fillna(1)
    soh['Commited'] = soh['Commited'].astype(float)*soh['Conversion Factor'].astype(float)
    soh['Available'] = soh['Available'].astype(float)*soh['Conversion Factor'].astype(float)
    
    
    whs = soh[(soh["WHS"]=="164")|(soh["WHS"]=="166")|(soh["WHS"]=="168")]
    def lessThan(row):
        if row['Available'] < 0:
            return "LESS"
        else:
            return "NA"
    whs['LessThan'] = whs.apply(lessThan, axis=1)
    whs = whs[whs['LessThan'] != 'LESS']
    
    el = ['164','166','168']
    dfs = []
    for i in el:
        df = whs[whs['WHS']==i]
        dfs.append(df)
        
    for i in range(len(dfs)):
        def classify(row):
            if row['Location'] == "R":
                return "Rec_{}".format(el[i])
            elif row['Location'] == 'Q' or row['Location'] == "H"  or row['Location'] == "HS":
                return "QA_{}".format(el[i])
            else:
                return "Avail_{}".format(el[i])
        dfs[i]['Class'] = dfs[i].apply(classify,axis=1)
        
    whsDF = pd.concat(dfs, ignore_index=True)
    whsDF = whsDF.pivot_table(index="Item Number", columns="Class", values="Available",aggfunc=np.sum).rename_axis(index=None, columns=None)
    whsDF = whsDF.fillna(0)
    whsDF.reset_index(inplace=True)
    whsDF = whsDF.rename(columns={"index":"Item Number"})
    
    
    sohb = soh[(soh["WHS"]!="164")&(soh["WHS"]!="166")&(soh["WHS"]!="168")]
    sohb = sohb[~sohb['WHS'].str.startswith("Total")]
    
    sohb['LessThan'] = sohb.apply(lessThan, axis=1)
    sohb = sohb[sohb['LessThan'] != 'LESS']
    sohb = sohb.pivot_table(index="Item Number", columns="WHS", values="Available",aggfunc=np.sum).rename_axis(index=None, columns=None)
    
    sohb.reset_index(inplace=True)
    sohb = sohb.rename(columns={"index":"Item Number"})
    sohb = sohb.fillna(0)
    
    
    return whsDF,sohb

whsDF,sohb = sohExtract(con)



def salesExtract(con):
    sales = pd.read_excel("UPDATE.xlsx", sheet_name="6MSALES")
    sales =  sales.drop("Sales Cat Code", axis=1)
    sales = pd.merge(sales, con[['Item Number',"Conversion Factor"]], on='Item Number', how='left')
    sales['Conversion Factor'] = sales['Conversion Factor'].fillna(1)
    sales.iloc[:,1:-1]=sales.iloc[:,1:-1].multiply(sales.iloc[:,-1], axis=0)
    sales =  sales.drop("Conversion Factor", axis=1)
    sales['6mAvg'] = round(sales.iloc[:, -6:].mean(axis=1),)
    return sales

#sales = salesExtract(con)

def rbuSales(con):
    
    allsales = pd.read_excel("UPDATE.xlsx", sheet_name="RBUSALES")
    allsales['Item Number'] = allsales['Item Number'].astype(str)
    allsales = pd.merge(allsales, con[['Item Number',"Conversion Factor"]], on='Item Number', how='left')
    allsales['Conversion Factor'] = allsales['Conversion Factor'].fillna(1)
    allsales.iloc[:,3:-1]=allsales.iloc[:,3:-1].multiply(allsales.iloc[:,-1], axis=0)
    allsales =  allsales.drop("Conversion Factor", axis=1)
    allsales = allsales[(allsales['Warehouse']==164)|(allsales['Warehouse']==166)|(allsales['Warehouse']==168)]
    
    sales = allsales.groupby('Item Number').sum().iloc[:, -6:]
    sales.reset_index(inplace=True)
    sales['6mAvg'] = round(sales.iloc[:, -6:].mean(axis=1),)
    
    
    col = list(allsales.columns[-6:])
    tsales = allsales.groupby(['Warehouse','Item Number']).sum().iloc[:, -6:]
    tsales.reset_index(inplace=True)
    warehouse_164 = pd.DataFrame()
    warehouse_166 = pd.DataFrame()
    warehouse_168 = pd.DataFrame()

    # Filter rows for each warehouse and copy the "Item Number" column
    warehouse_164['Item Number'] = tsales[tsales['Warehouse'] == 164]['Item Number'].values
    warehouse_166['Item Number'] = tsales[tsales['Warehouse'] == 166]['Item Number'].values
    warehouse_168['Item Number'] = tsales[tsales['Warehouse'] == 168]['Item Number'].values

    # Loop through the rows of the initial dataframe and populate the new dataframes
    for month in col:
        warehouse_164[f"164_{month}"] = tsales[tsales['Warehouse'] == 164][month].values
        warehouse_166[f"166_{month}"] = tsales[tsales['Warehouse'] == 166][month].values
        warehouse_168[f"168_{month}"] = tsales[tsales['Warehouse'] == 168][month].values


    warehouse_164.reset_index(drop=True, inplace=True)
    warehouse_166.reset_index(drop=True, inplace=True)
    warehouse_168.reset_index(drop=True, inplace=True)
    
    

    
    return allsales,sales, warehouse_164,warehouse_166,warehouse_168

allsales,sales, warehouse_164,warehouse_166,warehouse_168 = rbuSales(con)


def previousBO(con,fors):
    pbo = pd.read_excel("UPDATE.xlsx",sheet_name="PBO")
    pbo['Item Number'] = pbo['Item Number'].astype(str)
    pbo = pd.merge(pbo, con[['Item Number',"Conversion Factor"]], on='Item Number', how='left')
    pbo['Conversion Factor'] = pbo['Conversion Factor'].fillna(1)
    pbo.iloc[:,4:-1]=pbo.iloc[:,4:-1].multiply(pbo.iloc[:,-1], axis=0)
    pbo =  pbo.drop("Conversion Factor", axis=1)
    
    pItemBO = pbo.groupby('Item Number')[pbo.columns[4:]].sum()
    pItemBO.reset_index(inplace=True)
    
    pwItemBO = pbo.groupby(["Warehouse",'Item Number'])[pbo.columns[4:]].sum()
    pwItemBO.reset_index(inplace=True)
    pwItemBO['Warehouse'] = pwItemBO['Warehouse'].astype(int)
    col = pwItemBO.columns[-3:]
    
    warehouse_164y = pd.DataFrame()
    warehouse_166y = pd.DataFrame()
    warehouse_168y = pd.DataFrame()

    # Filter rows for each warehouse and copy the "Item Number" column
    warehouse_164y['Item Number'] = pwItemBO[pwItemBO['Warehouse'] == 164]['Item Number'].values
    warehouse_166y['Item Number'] = pwItemBO[pwItemBO['Warehouse'] == 166]['Item Number'].values
    warehouse_168y['Item Number'] = pwItemBO[pwItemBO['Warehouse'] == 168]['Item Number'].values

    # Loop through the rows of the initial dataframe and populate the new dataframes
    for month in col:
        warehouse_164y[f"164_{month}"] = pwItemBO[pwItemBO['Warehouse'] == 164][month].values
        warehouse_166y[f"166_{month}"] = pwItemBO[pwItemBO['Warehouse'] == 166][month].values
        warehouse_168y[f"168_{month}"] = pwItemBO[pwItemBO['Warehouse'] == 168][month].values


    warehouse_164y.reset_index(drop=True, inplace=True)
    warehouse_166y.reset_index(drop=True, inplace=True)
    warehouse_168y.reset_index(drop=True, inplace=True)
    
    
    pRItemBO = pd.merge(pbo, fors[['Item Number',"Sls Cd3"]], on='Item Number', how='left')
    pRItemBO['KEY3'] = pRItemBO['RBU'].astype(str)+"_"+pRItemBO['Sls Cd3'].astype(str)+"_"+pRItemBO['Item Number']
    pRItemBO.drop(columns=['KEY','RBU','Item Number','Sls Cd3'], inplace=True)
    
    pRBUBO = pRItemBO.groupby("KEY3")[pRItemBO.columns[-4:-1]].sum()
    pRBUBO.reset_index(inplace=True)
    col = pRItemBO.columns[-4:-1]
    
    warehouse_164z = pd.DataFrame()
    warehouse_166z = pd.DataFrame()
    warehouse_168z = pd.DataFrame()

    # Filter rows for each warehouse and copy the "Item Number" column
    warehouse_164z['KEY3'] = pRItemBO[pRItemBO['Warehouse'] == 164]['KEY3'].values
    warehouse_166z['KEY3'] = pRItemBO[pRItemBO['Warehouse'] == 166]['KEY3'].values
    warehouse_168z['KEY3'] = pRItemBO[pRItemBO['Warehouse'] == 168]['KEY3'].values

    # Loop through the rows of the initial dataframe and populate the new dataframes
    for month in col:
        warehouse_164z[f"164_{month}"] = pRItemBO[pRItemBO['Warehouse'] == 164][month].values
        warehouse_166z[f"166_{month}"] = pRItemBO[pRItemBO['Warehouse'] == 166][month].values
        warehouse_168z[f"168_{month}"] = pRItemBO[pRItemBO['Warehouse'] == 168][month].values


    warehouse_164z.reset_index(drop=True, inplace=True)
    warehouse_166z.reset_index(drop=True, inplace=True)
    warehouse_168z.reset_index(drop=True, inplace=True)
    
    
    return pbo, pItemBO,warehouse_164y,warehouse_166y,warehouse_168y,pRBUBO,warehouse_164z,warehouse_166z,warehouse_168z

pbo, pItemBO,warehouse_164y,warehouse_166y,warehouse_168y,pRBUBO,warehouse_164z,warehouse_166z,warehouse_168z = previousBO(con,fors)


def sit_prep(con):
    sit = pd.read_excel("UPDATE.xlsx", sheet_name="SIT")
    sit = sit[["Item Number","P/O Bus Unit","ST Shipped Qty"]]
    sit['P/O Bus Unit'] = sit['P/O Bus Unit'].astype(int)
    sit = sit[(sit["P/O Bus Unit"]==164)|(sit["P/O Bus Unit"]==166)|(sit["P/O Bus Unit"]==168)]
    sit = sit.pivot_table(index="Item Number", columns="P/O Bus Unit", values="ST Shipped Qty",aggfunc=np.sum).rename_axis(index=None, columns=None)
    sit = sit.fillna(0)
    sit = sit.round(0)
    sit.reset_index(inplace=True)
    sit.rename(columns = {"index":"Item Number",164:"ST_164",166:"ST_166",168:"ST_168"}, inplace = True)
    sit = pd.merge(sit, con[['Item Number','Conversion Factor']], on='Item Number',how='left')
    sit['Conversion Factor'] = sit['Conversion Factor'].fillna(1)
    sit.iloc[:,1:-1]=sit.iloc[:,1:-1].multiply(sit.iloc[:,-1], axis=0)
    sit =  sit.drop("Conversion Factor", axis=1)
    return sit

sit = sit_prep(con)


def rbuMTD(mtd):
    rbuMTD = mtd[['KEY3','Branch/\nPlant','QtyShipped']]
    rbuMTD = rbuMTD.rename(columns={"Branch/\nPlant":"Warehouse"})
    rbuMTD["Warehouse"] = rbuMTD["Warehouse"].astype(int)
    rbuMTD["Warehouse"] = rbuMTD["Warehouse"].astype(str)
    rbuMTD = rbuMTD[(rbuMTD['Warehouse']=='164')|(rbuMTD['Warehouse']=='166')|(rbuMTD['Warehouse']=='168')]
    rbuMTD = rbuMTD.pivot_table(index="KEY3", columns="Warehouse", values="QtyShipped",aggfunc=np.sum).rename_axis(index=None, columns=None)
    rbuMTD = rbuMTD.fillna(0)
    rbuMTD.reset_index(inplace=True)
    rbuMTD.rename(columns = {"index":"KEY3",'164':"Mtd_164",'166':"Mtd_166",'168':"Mtd_168"}, inplace = True)
    return rbuMTD

rbuMTD = rbuMTD(mtd)

def finalPrep(test,wo,orders,whsDF,sohb,sales,pfc,sit,warehouse_164,warehouse_166,warehouse_168,fors,rbuMTD,pItemBO,warehouse_164y,warehouse_166y,warehouse_168y):
    
    bo = orders[["Item Number","TotalBackOrders","Branch/\nPlant"]]
    bo['WHS'] = bo['Branch/\nPlant'].str.rsplit(' ',expand=True)[0]
    bo = bo[(bo["WHS"]=="164")|(bo["WHS"]=="166")|(bo["WHS"]=="168")]
    bo = bo.pivot_table(index="Item Number", columns="WHS", values="TotalBackOrders",aggfunc=np.sum).rename_axis(index=None, columns=None)
    bo.reset_index(inplace=True)
    bo = bo.rename(columns={"index":'Item Number'})
    bo = bo.fillna(0)
    bo = bo.rename(columns={"164":"BO_164","166":"BO_166","168":"BO_168"})
    
    final = test.groupby(['Item Number',"SalesCode"]).apply(lambda x: x.iloc[:, 4:17].sum())
    final.reset_index(inplace=True)
    final = pd.merge(final,sales, on='Item Number', how='outer')
    final = pd.merge(final,warehouse_164, on='Item Number', how='outer')
    final = pd.merge(final,warehouse_166, on='Item Number', how='outer')
    final = pd.merge(final,warehouse_168, on='Item Number', how='outer')
    
    last = final.iloc[:,-25:]
    
    final = final.iloc[:, :-25]
    final = pd.concat([final.iloc[:, :3], last, final.iloc[:, 3:]], axis=1)
    
    final = pd.merge(final, pfc, on='Item Number', how='outer')
    
    last = final.iloc[:,-3:]
    final = final.iloc[:, :-3]
    final = pd.concat([final.iloc[:, :28], last, final.iloc[:, 28:]], axis=1)
    
    
    final = pd.merge(final,bo,on='Item Number',how='outer')
    final = pd.merge(final,whsDF, on='Item Number', how='outer')
    final = pd.merge(final,sit, on='Item Number', how='outer')
    final = pd.merge(final,sohb, on='Item Number', how='left')
    final = pd.merge(final,wo,on='Item Number', how='outer')
    
    fors = fors[["Item Number",'Plan Fmly',"Sls Cd4","ABC 1 Sls", 'Safety Stk 164', 'Safety Stk 166','Safety Stk 168']]
    fors = fors.rename(columns={"Safety Stk 164":"SS_164","Safety Stk 166":"SS_166","Safety Stk 168":"SS_168"})
    
    rbuMTD[['RBU', 'SalesCode','Item Number']] = rbuMTD['KEY3'].str.split(pat='_', n=2, expand=True)
    rbuMTD = rbuMTD.groupby('Item Number')[['Mtd_164', 'Mtd_166', 'Mtd_168']].sum().reset_index()
    rbuMTD['Item Number'] = rbuMTD['Item Number'].astype(str)
    final = pd.merge(final,rbuMTD, on='Item Number', how='left')
    
    final = pd.merge(final,pItemBO, on='Item Number', how='left')
    final = pd.merge(final,warehouse_164y, on='Item Number', how='left')
    final = pd.merge(final,warehouse_166y, on='Item Number', how='left')
    final = pd.merge(final,warehouse_168y, on='Item Number', how='left')

    
    final = pd.merge(final,fors, on='Item Number', how='left')
    final = final.fillna(0)
    return final

final = finalPrep(test,wo,orders,whsDF,sohb,sales,pfc,sit,warehouse_164,warehouse_166,warehouse_168,fors,rbuMTD,pItemBO,warehouse_164y,warehouse_166y,warehouse_168y)



def rbuWarehouse(allsales,fors):
    allsales = pd.merge(allsales,fors[['Item Number',"Sls Cd3"]], on='Item Number',how='left')
    allsales['Sls Cd3'] = allsales['Sls Cd3'].replace(' ', np.nan)
    allsales['Sls Cd3'] = allsales['Sls Cd3'].astype(float).astype(pd.Int64Dtype())
    allsales['KEY3'] = allsales['RBU'].astype(str)+"_"+allsales['Sls Cd3'].astype(str)+"_"+allsales['Item Number'].astype(str)
    
    rbuSALES = allsales
    rbuSALES = rbuSALES.rename(columns={"Sls Cd3":"SalesCode"})
    rbuSALESx = rbuSALES.drop(columns=['RBU','Item Number','SalesCode'])
    last_column = rbuSALESx.columns[-1]
    rbuSALESx = rbuSALESx[[last_column] + [col for col in rbuSALESx.columns if col != last_column]]
    
    
    col = list(rbuSALESx.columns[-6:])

    warehouse_164x = pd.DataFrame()
    warehouse_166x = pd.DataFrame()
    warehouse_168x = pd.DataFrame()

    # Filter rows for each warehouse and copy the "Item Number" column
    warehouse_164x['KEY3'] = rbuSALESx[rbuSALESx['Warehouse'] == 164]['KEY3'].values
    warehouse_166x['KEY3'] = rbuSALESx[rbuSALESx['Warehouse'] == 166]['KEY3'].values
    warehouse_168x['KEY3'] = rbuSALESx[rbuSALESx['Warehouse'] == 168]['KEY3'].values

    # Loop through the rows of the initial dataframe and populate the new dataframes
    for month in col:
        warehouse_164x[f"164_{month}"] = rbuSALESx[rbuSALESx['Warehouse'] == 164][month].values
        warehouse_166x[f"166_{month}"] = rbuSALESx[rbuSALESx['Warehouse'] == 166][month].values
        warehouse_168x[f"168_{month}"] = rbuSALESx[rbuSALESx['Warehouse'] == 168][month].values


    warehouse_164x.reset_index(drop=True, inplace=True)
    warehouse_166x.reset_index(drop=True, inplace=True)
    warehouse_168x.reset_index(drop=True, inplace=True)

    
    return rbuSALES,warehouse_164x,warehouse_166x,warehouse_168x

rbuSALES,warehouse_164x,warehouse_166x,warehouse_168x = rbuWarehouse(allsales,fors)


def rbuORDERS(orders):
    rbuORDERS = orders[['BU\nHeader',"Sales Reporting Code 03","Item Number",'Branch/\nPlant',
                        'TotalOpenOrders',"TotalBackOrders","PastDue_BO",'CurrDue_BO']]
    rbuORDERS[['Warehouse', 'PLANTNAME']] = rbuORDERS['Branch/\nPlant'].str.split(pat='-', n=1, expand=True)
    rbuORDERS['Warehouse'] = rbuORDERS['Warehouse'].str.strip()
    rbuORDERS['Warehouse'] = rbuORDERS['Warehouse'].astype(int)
    rbuORDERS['Warehouse'] = rbuORDERS['Warehouse'].astype(str)
    rbuORDERS = rbuORDERS.rename(columns={"BU\nHeader":"RBU","Sales Reporting Code 03":"SalesCode"})
    rbuORDERS.drop(columns=['PLANTNAME','Branch/\nPlant'],inplace=True)
    rbuORDERS['RBU'] = rbuORDERS['RBU'].astype(int)
    rbuORDERS['SalesCode'] = rbuORDERS['SalesCode'].astype(int)
    rbuORDERS['Item Number'] = rbuORDERS['Item Number'].astype(str)
    rbuORDERS['KEY3'] = rbuORDERS['RBU'].astype(str)+"_"+rbuORDERS['SalesCode'].astype(str)+"_"+rbuORDERS['Item Number'].astype(str)
    last_column = rbuORDERS.columns[-1]
    rbuORDERS = rbuORDERS[[last_column] + [col for col in rbuORDERS.columns if col != last_column]]
    
    wOrders = rbuORDERS[['KEY3','TotalOpenOrders','Warehouse']].pivot_table(index="KEY3", columns="Warehouse", values="TotalOpenOrders",aggfunc=np.sum).rename_axis(index=None, columns=None)
    wCurrBO = rbuORDERS[['KEY3','CurrDue_BO','Warehouse']].pivot_table(index="KEY3", columns="Warehouse", values="CurrDue_BO",aggfunc=np.sum).rename_axis(index=None, columns=None)
    wPastBO = rbuORDERS[['KEY3','PastDue_BO','Warehouse']].pivot_table(index="KEY3", columns="Warehouse", values="PastDue_BO",aggfunc=np.sum).rename_axis(index=None, columns=None)
    
    wOrders = wOrders.fillna(0)
    wCurrBO = wCurrBO.fillna(0)
    wPastBO = wPastBO.fillna(0)

    wOrders.reset_index(inplace=True)
    wCurrBO.reset_index(inplace=True)
    wPastBO.reset_index(inplace=True)

    wOrders.rename(columns = {"index":"KEY3",'164':"OpenOrders_164",'166':"OpenOrders_166",'168':"OpenOrders_168"}, inplace = True)
    wCurrBO.rename(columns = {"index":"KEY3",'164':"CurrBO_164",'166':"CurrBO_166",'168':"CurrBO_168"}, inplace = True)
    wPastBO.rename(columns = {"index":"KEY3",'164':"PastBO_164",'166':"PastBO_166",'168':"PastBO_168"}, inplace = True)
    
    return wOrders,wCurrBO,wPastBO
    
wOrders,wCurrBO,wPastBO = rbuORDERS(orders)





def rbuSummary(test,ofc,rbuSALES,warehouse_164x,warehouse_166x,warehouse_168x,lfc,acc,wOrders,wCurrBO,wPastBO,rbuMTD,pRBUBO,warehouse_164z,warehouse_166z,warehouse_168z):
    
    rbuSALESy = rbuSALES.groupby(['KEY3',"RBU",'Item Number','SalesCode']).apply(lambda x: x.iloc[:, 3:9].sum())
    rbuSALESy.reset_index(inplace=True)
    
    test = pd.merge(test, rbuSALESy, on='KEY3', how='outer',suffixes=('', '_y'))
    test['Item Number'].fillna(test['Item Number_y'], inplace=True)
    test['SalesCode'].fillna(test['SalesCode_y'], inplace=True)
    test['RBU'].fillna(test['RBU_y'], inplace=True)
    test.drop(test.filter(regex='_y$').columns, axis=1, inplace=True)
    test['RBU']= test['RBU'].astype(int)
    
    test = pd.merge(test, lfc, on='KEY3', how='outer',suffixes=('', '_y'))
    test['Item Number'].fillna(test['Item Number_y'], inplace=True)
    test['SalesCode'].fillna(test['SalesCode_y'], inplace=True)
    test['RBU'].fillna(test['RBU_y'], inplace=True)
    test.drop(test.filter(regex='_y$').columns, axis=1, inplace=True)
    test['RBU']= test['RBU'].astype(int)
    
    last = test.iloc[:,-3:]
    test = test.iloc[:, :-3]
    test = pd.concat([test.iloc[:, :5], last, test.iloc[:, 5:]], axis=1)
    
    test = pd.merge(test,warehouse_164x, on='KEY3', how='left')
    test = pd.merge(test,warehouse_166x, on='KEY3', how='left')
    test = pd.merge(test,warehouse_168x, on='KEY3', how='left')
    
    test = pd.merge(test,rbuMTD, on='KEY3', how='left',suffixes=('', '_y'))
    test['Item Number'].fillna(test['Item Number_y'], inplace=True)
    test['SalesCode'].fillna(test['SalesCode_y'], inplace=True)
    test['RBU'].fillna(test['RBU_y'], inplace=True)
    test.drop(test.filter(regex='_y$').columns, axis=1, inplace=True)
    test['RBU']= test['RBU'].astype(int)
    
    test = pd.merge(test,wOrders, on='KEY3', how='left')
    test = pd.merge(test,wCurrBO, on='KEY3', how='left')
    test = pd.merge(test,wPastBO, on='KEY3', how='left')
    
    
    rbu = acc[['GROUP','DIVISION','RBU','SalesCode']]
    rbu = rbu.drop_duplicates(subset=['GROUP','DIVISION','RBU','SalesCode'], keep='last')
    rbu['SalesCode'] = rbu['SalesCode'].replace(' ', np.nan)
    rbu['SalesCode'] = rbu['SalesCode'].astype(float).astype(pd.Int64Dtype())
    rbu['RBU'] = rbu['RBU'].replace(' ', np.nan)
    rbu['RBU'] = rbu['RBU'].astype(float).astype(pd.Int64Dtype())
    
    
    
    rbu['KEY4'] = rbu['RBU'].astype(str)+"_"+rbu['SalesCode'].astype(str)


    #test['RBU'] = test['RBU'].replace(' ', np.nan)
    #test['RBU'] = test['RBU'].astype(float).astype(pd.Int64Dtype())
    test['KEY4'] = test['RBU'].astype(str)+"_"+test['SalesCode'].astype(str)

    test.drop(columns=['GROUP','DIVISION'], inplace=True)

    test = pd.merge(test,rbu[['KEY4','GROUP','DIVISION']], on='KEY4',how='left')
    
    test = pd.merge(test,pRBUBO, on='KEY3', how='left')
    test = pd.merge(test,warehouse_164z, on='KEY3', how='left')
    test = pd.merge(test,warehouse_166z, on='KEY3', how='left')
    test = pd.merge(test,warehouse_168z, on='KEY3', how='left')
    
    test = test.drop_duplicates(subset='KEY3', keep='last')
    
    test = test.fillna(0)
    
    return test

test = rbuSummary(test,ofc,rbuSALES,warehouse_164x,warehouse_166x,warehouse_168x,lfc,acc,wOrders,wCurrBO,wPastBO,rbuMTD,pRBUBO,warehouse_164z,warehouse_166z,warehouse_168z)

cols = ['QA_164','QA_166','QA_168',"Rec_164","Rec_168","Rec_166","ST_164","ST_166","ST_168"]

for i in cols:
    if i in list(final.columns):
        pass
    else:
        final[i] = 0
        
def indicator(fc,final):
    ind = pd.read_excel("FILE.xlsx",sheet_name="INDICATOR")
    ind['SignalC'] = ind['SignalC'].astype(str)
    final['Plan Fmly'] = final['Plan Fmly'].astype(str)
    ind['ColC'] = list(fc.columns)[2]
    ind = ind[ind['GROUP']=="A1"]
    dfs = []
    dfs = pd.DataFrame(dfs)
    dfs['Item Number'] = final['Item Number'].astype(str)
    def A(row):
        A = ind[ind['CAT']=="A"]

        colC_value = list(A['ColC'])[0]

        plan_fmly = row['Plan Fmly']

        bo = row['TotalBackOrders']

        if plan_fmly == "DIS" and bo > 0:
            return list(A[A['SignalC']=="FC = 0"]['Status'])[0]
        else:
            return 0

    def B(row):
        B = ind[ind['CAT']=="B"]

        colC_value = list(B['ColC'])[0]

        plan_fmly = row['Plan Fmly']

        bo = row['TotalBackOrders']

        if row[colC_value] == 0 and plan_fmly == "980" and bo > 0:
            return list(B[B['SignalC']=="FC = 0"]['Status'])[0]
        else:
            return 0

    def C(row):
        C = ind[ind['CAT']=="C"]

        colC_value = list(C['ColC'])[0]

        bypass = row['WO ByPass']

        bo = row['TotalBackOrders']

        if bypass > 0 and bo > 0:
            return list(C[C['SignalC']=="FC / D"]['Status'])[0]
        else:
            return 0

    def D(row):
        D = ind[ind['CAT']=="D"]

        colC_value = list(D['ColC'])[0]

        bypass = row['WO ByPass']

        bo = row['TotalBackOrders']

        if row[colC_value] == 0 and bypass == 0 and bo > 0:
            return list(D[D['SignalC']=="FC = 0"]['Status'])[0]
        elif row[colC_value] > (row['TotalOpenOrders']+row['QtyShipped']) and bypass == 0 and bo > 0:
            return list(D[D['SignalC']=="FC > D"]['Status'])[0]
        elif row[colC_value] < (row['TotalOpenOrders']+row['QtyShipped']) and bypass == 0 and bo > 0:
            return list(D[D['SignalC']=="FC < D"]['Status'])[0]
        else:
            return 0


    dfs['A'] = final.apply(A, axis=1)
    dfs['B'] = final.apply(B, axis=1)
    dfs['C'] = final.apply(C, axis=1)
    dfs['D'] = final.apply(D, axis=1)
    
    def Issue(row):
        if row['A'] == 0:
            if row['B'] == 0:
                if row['C'] == 0:
                    if row['D'] == 0:
                        return 0
                    else:
                        return row['D']
                else:
                    return row['C']
            else:
                return row['B']
        else:
            return row['A']
                   
    dfs['BO_CommentA'] = dfs.apply(Issue, axis=1)
    
    final = pd.merge(final,dfs[['Item Number','BO_CommentA']], on='Item Number', how='left')
                           
    
    return final

final = indicator(fc,final)


def indicator2(final):
    ind = pd.read_excel("FILE.xlsx",sheet_name="INDICATOR")
    ind['SignalC'] = ind['SignalC'].astype(str)
    #final['Plan Fmly'] = final['Plan Fmly'].astype(str)
    ind['ColC'] = list(fc.columns)[2]
    ind = ind[ind['GROUP']=="B1"]
    final['3 >= ZeroSales'] = (final.iloc[:,3:9].astype(int) == 0).sum(axis=1) >= 3
    final['std'] = np.std(final.iloc[:,3:9],axis=1)
    dfs = []
    dfs = pd.DataFrame(dfs)
    dfs['Item Number'] = final['Item Number'].astype(str)
    def A(row):
        A = ind[ind['CAT']=="A"]

        colC_value = list(A['ColC'])[0]


        commentA = row['BO_CommentA']
        pareto = row['ABC 1 Sls']

        std = row['std']

        avg = row['6mAvg']

        mtd = row['QtyShipped']+row['TotalOpenOrders']

        zeroSales = row['3 >= ZeroSales']

        if commentA != "0" and pareto == 'D' and (std+avg) < mtd:
            return list(A[A['SignalC']=="6mA+std > mtd"]['Status'])[0]
        if commentA != "0" and pareto == 'D' and zeroSales == True:
            return list(A[A['SignalC']=="3 >= ZeroSales"]['Status'])[0]
        else:
            return commentA
    
    dfs['BO_CommentA'] = final.apply(A, axis=1)
    
    final = final.drop('BO_CommentA',axis=1)
    
    final = pd.merge(final,dfs[['Item Number','BO_CommentA']], on='Item Number', how='left')
    
    
    dfs = []
    dfs = pd.DataFrame(dfs)
    dfs['Item Number'] = final['Item Number'].astype(str)
    
    def B(row):
        B = ind[ind['CAT']=="B"]

        colC_value = list(B['ColC'])[0]


        commentA = row['BO_CommentA']
        pareto = row['ABC 1 Sls']

        std = row['std']

        avg = row['6mAvg']

        mtd = row['QtyShipped']+row['TotalOpenOrders']

        zeroSales = row['3 >= ZeroSales']

        if commentA == "Demand Exceed Forecast" and pareto != 'D' and (std+avg) < mtd:
            return list(B[B['SignalC']=="6mA+std > mtd"]['Status'])[0]
        if commentA == "Demand Exceed Forecast" and pareto != 'D' and zeroSales == True:
            return list(B[B['SignalC']=="3 >= ZeroSales"]['Status'])[0]
        else:
            return commentA
    
    dfs['BO_CommentA'] = final.apply(B, axis=1)

    final = final.drop('BO_CommentA',axis=1)
    
    final = pd.merge(final,dfs[['Item Number','BO_CommentA']], on='Item Number', how='left')
    
    final = final.drop_duplicates(subset='Item Number', keep='first')
    
    return final
    
final = indicator2(final)

def publishExcelFile(acc,UnRegistered, UnRegisteredOrders,test,final):
    excel_file = 'DashFile.xlsx'
    book = load_workbook(excel_file)


    with pd.ExcelWriter(excel_file, engine='openpyxl', mode='a',if_sheet_exists='replace') as writer:
        writer.book = book


        if 'CustomerSales' in writer.book.sheetnames:
            sheet = writer.book['CustomerSales']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)


        acc.to_excel(writer, sheet_name='CustomerSales', index=False)
        

        if 'UnregisteredSales' in writer.book.sheetnames:
            sheet = writer.book['UnregisteredSales']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)

        UnRegistered.to_excel(writer, sheet_name='UnregisteredSales', index=False)


        if 'UnregisteredOrders' in writer.book.sheetnames:
            sheet = writer.book['UnregisteredOrders']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)

        UnRegisteredOrders.to_excel(writer, sheet_name='UnregisteredOrders', index=False)

        if 'RBUSummary' in writer.book.sheetnames:
            sheet = writer.book['RBUSummary']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)

        test.to_excel(writer, sheet_name='RBUSummary', index=False)

        if 'ItemNumbers' in writer.book.sheetnames:
            sheet = writer.book['ItemNumbers']
            sheet.delete_rows(sheet.min_row + 1, sheet.max_row)

        final.to_excel(writer, sheet_name='ItemNumbers', index=False)



    book.save(excel_file)
    

publishExcelFile(acc,UnRegistered, UnRegisteredOrders,test,final)


    

WHYY
   7.1
   7.2
   7.3
   7.4


C:\Users\gwalal\AppData\Local\Temp\ipykernel_30172\1893965733.py:1027: FutureWarning: Setting the `book` attribute is not part of the public API, usage can give unexpected or corrupted results and will be removed in a future version
  writer.book = book
